In [122]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import WorkerOptions
import apache_beam as beam
import time
from apache_beam.runners import DataflowRunner


options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True
options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#number of workers
options.view_as(WorkerOptions).num_workers = 2

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)




In [ ]:
########################################################################  With A Single Worker    ############################################################################


query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records`'



def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))

def makeRow(x):
    row = dict(zip(('id','name'),x))
    return row

class make_api_call(beam.DoFn):
    
    def process(self, element):
        # importing the requests library
        import requests
        import json
        import urllib.request
        #from kafka import KafkaProducer
        
        #producer = KafkaProducer(bootstrap_servers=['34.28.118.32:9094'], api_version=(0, 10))
        
        #content = urllib.request.urlopen('http://35.239.64.67:32726/').read().decode('utf-8')
        #row = dict(zip(('message'),content))
        
        
        
        # The API endpoint
        #url = "http://35.239.64.67:32726/"
        url = "https://jsonplaceholder.typicode.com/posts/1"

        # A GET request to the API
        response = requests.get(url)

        # Print the response
        response_json = response.json()
        #producer.send('my-topic', json.dumps(response_json).encode('utf-8'))
        #if producer is not None:
                    #producer.close()

        length = len(element[1])
        x = response_json['userId']
        x1 = response_json['id']
        x2 = response_json['title']
        x3 = response_json['body']
        #x=1
        #x1=1
        #x2='string'
        #x3='strong'
        
        row = dict(zip(('userId','id','title','body','length'),(x,x1,x2,x3,length)))
        yield row #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator
          

from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

p = beam.Pipeline(options=options)

#with beam.Pipeline(options=options) as p:
read_requests =(p| 
    beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )
    |beam.Map(insert_key)
    |'Group into batches' >> beam.GroupIntoBatches(50000)
    |'Make an External API Call' >> beam.ParDo(make_api_call())
    |"Write to Bigquery">>beam.io.Write(beam.io.WriteToBigQuery(        'my-table-message-second',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='userId:INTEGER, id:INTEGER,title:STRING,body:STRING,length:INTEGER',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )
    )
    #|beam.Map(print))
               
pipeline_result = DataflowRunner().run_pipeline(p, options=options)

In [33]:

query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records` LIMIT 10'



def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))

def cross_join(left, rights):
    for x in rights:
        yield (left, x)

from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

#p = beam.Pipeline(options=options)

with beam.Pipeline(options=options) as p:
    main_input = p|beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )|beam.Map(insert_key)#|beam.Map(print))
    
    side_input = (p|"Side Input From GCS" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/inputs_file01.csv'))
    
    result = (main_input|"Cross join">>beam.FlatMap(cross_join, rights=beam.pvalue.AsIter(side_input)))|beam.Map(print) 

(('INSBI7', ('INSBI759226449', 'hthEUEqmx')), '10.013411,200.088149,500.609548,3.174919,-10999,14.087415')
(('INSBI7', ('INSBI759226449', 'hthEUEqmx')), '4.013411,2.088149,5.661589,3.174919,-99999,4.087415')
(('INSBI6', ('INSBI652587880', 'XqMkSHEva')), '10.013411,200.088149,500.609548,3.174919,-10999,14.087415')
(('INSBI6', ('INSBI652587880', 'XqMkSHEva')), '4.013411,2.088149,5.661589,3.174919,-99999,4.087415')
(('INSBI5', ('INSBI577879924', 'noNLOvNIo')), '10.013411,200.088149,500.609548,3.174919,-10999,14.087415')
(('INSBI5', ('INSBI577879924', 'noNLOvNIo')), '4.013411,2.088149,5.661589,3.174919,-99999,4.087415')
(('INSBI1', ('INSBI188022073', 'jhpicJqUN')), '10.013411,200.088149,500.609548,3.174919,-10999,14.087415')
(('INSBI1', ('INSBI188022073', 'jhpicJqUN')), '4.013411,2.088149,5.661589,3.174919,-99999,4.087415')
(('INSBI2', ('INSBI230887749', 'CajgKklNk')), '10.013411,200.088149,500.609548,3.174919,-10999,14.087415')
(('INSBI2', ('INSBI230887749', 'CajgKklNk')), '4.013411,2.088

In [69]:

query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records` LIMIT 20'


'''
def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))
'''
'''
def cross_join(left, rights):
    for x in rights:
        yield (left, x)
'''        
        
'''        
def singleton_cross_join(left, rights):
    return (left, rights)
'''

def cross_join(left, rights):
    for x in rights:
        yield (left['id'],left['name'],x)
        
def insert_key(x):
    return (x[0][0:6],(x[0],x[1],x[2]))



class make_api_call(beam.DoFn):
    
    def process(self, element):
        # importing the requests library
        import requests
        import json
        import urllib.request
        
        url_post = 'http://35.192.38.85:80/hello/post'
        data_dict={}
        data_dict['customer_list']=element
        post_response = requests.post(url_post, data = data_dict)
        #data_dict['customer_name_list']=element[1][1]
        #data_dict['new_sanction_name_list']=element[1][2][0].split(',')
        
        # The API endpoint
        #url = "http://35.239.64.67:32726/"
        url = "https://jsonplaceholder.typicode.com/posts/1"

        # A GET request to the API
        response = requests.get(url)

        # Print the response
        response_json = response.json()
       
        length = len(element[1])
        x = response_json['userId']
        x1 = response_json['id']
        x2 = response_json['title']
        x3 = response_json['body']
        
        row = dict(zip(('userId','id','title','body','length'),(x,x1,x2,x3,length)))
        yield row #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator


from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

p = beam.Pipeline(options=options)

#with beam.Pipeline(options=options) as p:

main_input = (p|beam.io.ReadFromBigQuery(query=query,use_standard_sql=True ))#|beam.Map(insert_key)#|beam.Map(print))
    
side_input = (p|"Side Input From GCS" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/sanctioned_customer.txt'))
    
    #result = (main_input|"Cross join">>beam.FlatMap(singleton_cross_join, rights=beam.pvalue.AsSingleton(side_input)))|beam.Map(print) 
enriched = (main_input|"Cross join">>beam.FlatMap(cross_join, rights=beam.pvalue.AsIter(side_input)))#|beam.Map(print) 
key_insert = enriched|"Key Insertion">>beam.Map(insert_key)#|beam.Map(print)
batches = key_insert|'Group into batches' >> beam.GroupIntoBatches(5)|beam.Map(print)
'''
process = batches|'Make an External API Call' >> beam.ParDo(make_api_call())|"Write to Bigquery">>beam.io.Write(beam.io.WriteToBigQuery('my-table-message-second',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='userId:INTEGER, id:INTEGER,title:STRING,body:STRING',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )
    

'''
pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_3444/3443825875.py:78: FutureWarning: GroupIntoBatches is experimental.
  batches = key_insert|'Group into batches' >> beam.GroupIntoBatches(5)|beam.Map(print)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [73]:
query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records` LIMIT 10'



def cross_join(left, rights):
    for x in rights:
        yield (left['id'],left['name'],x)
        
def insert_key(x):
    return (x[0][0:6],(x[0],x[1],x[2]))

from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

#p = beam.Pipeline(options=options)

with beam.Pipeline(options=options) as p:
    #with beam.Pipeline(options=options) as p:
    main_input = (p|beam.io.ReadFromBigQuery(query=query,use_standard_sql=True ))#|beam.Map(insert_key)#|beam.Map(print))
    side_input = (p|"Side Input From GCS" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/sanctioned_customer.txt'))
    #result = (main_input|"Cross join">>beam.FlatMap(singleton_cross_join, rights=beam.pvalue.AsSingleton(side_input)))|beam.Map(print) 
    enriched = (main_input|"Cross join">>beam.FlatMap(cross_join, rights=beam.pvalue.AsIter(side_input)))#|beam.Map(print) 
    key_insert = enriched|"Key Insertion">>beam.Map(insert_key)|beam.Map(print)

('INSBI6', ('INSBI652587880', 'XqMkSHEva', 'yKEesKMCX,GjJFjdxaJ'))
('INSBI7', ('INSBI759226449', 'hthEUEqmx', 'yKEesKMCX,GjJFjdxaJ'))
('INSBI1', ('INSBI188022073', 'jhpicJqUN', 'yKEesKMCX,GjJFjdxaJ'))
('INSBI2', ('INSBI230887749', 'CajgKklNk', 'yKEesKMCX,GjJFjdxaJ'))
('INSBI5', ('INSBI585529195', 'xbbjDjkVx', 'yKEesKMCX,GjJFjdxaJ'))
('INSBI1', ('INSBI180433362', 'PvdkdMBYe', 'yKEesKMCX,GjJFjdxaJ'))
('INSBI4', ('INSBI481260153', 'ScQZUIHEv', 'yKEesKMCX,GjJFjdxaJ'))
('INSBI6', ('INSBI657731628', 'hNQKABNvb', 'yKEesKMCX,GjJFjdxaJ'))
('INSBI5', ('INSBI577879924', 'noNLOvNIo', 'yKEesKMCX,GjJFjdxaJ'))
('INSBI3', ('INSBI304740746', 'tadoHnbqL', 'yKEesKMCX,GjJFjdxaJ'))


In [77]:
query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records` LIMIT 10'



def cross_join(left, rights):
    for x in rights:
        yield (left['id'],left['name'],x)
        
def insert_key(x):
    return (x[0][0:6],(x[0],x[1],x[2]))

class make_api_call(beam.DoFn):
    
    def process(self, element):
        # importing the requests library
        import requests
        import json
        import urllib.request
        
        url_post = 'http://35.192.38.85:80/hello/post'
        data_dict={}
        data_dict['customer_list']=element
        post_response = requests.post(url_post, data = data_dict)
        #data_dict['customer_name_list']=element[1][1]
        #data_dict['new_sanction_name_list']=element[1][2][0].split(',')
        
        # The API endpoint
        #url = "http://35.239.64.67:32726/"
        url = "https://jsonplaceholder.typicode.com/posts/1"

        # A GET request to the API
        response = requests.get(url)

        # Print the response
        response_json = response.json()
       
        length = len(element[1])
        x = response_json['userId']
        x1 = response_json['id']
        x2 = response_json['title']
        x3 = response_json['body']
        
        row = dict(zip(('userId','id','title','body','length'),(x,x1,x2,x3,length)))
        yield row #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator


from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

#p = beam.Pipeline(options=options)

with beam.Pipeline(options=options) as p:
    #with beam.Pipeline(options=options) as p:
    main_input = (p|beam.Create([('INSBI652587880', 'XqMkSHEva', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI759226449', 'hthEUEqmx', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI188022073', 'jhpicJqUN', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI230887749', 'CajgKklNk', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI585529195', 'xbbjDjkVx', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI180433362', 'PvdkdMBYe', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI481260153', 'ScQZUIHEv', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI657731628', 'hNQKABNvb', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI577879924', 'noNLOvNIo', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI304740746', 'tadoHnbqL', 'yKEesKMCX,GjJFjdxaJ')
                                 ]))
    key_insert = main_input|"Key Insertion">>beam.Map(insert_key)| 'Group into batches' >> beam.GroupIntoBatches(2)|beam.Map(print)

/tmp/ipykernel_3444/1580589721.py:31: FutureWarning: GroupIntoBatches is experimental.
  key_insert = main_input|"Key Insertion">>beam.Map(insert_key)| 'Group into batches' >> beam.GroupIntoBatches(2)|beam.Map(print)
/jupyter/.kernels/apache-beam-2.44.0/lib/python3.8/site-packages/apache_beam/transforms/util.py:924: FutureWarning: GroupIntoBatches is experimental.
  return GroupIntoBatches(*_GroupIntoBatchesParams.parse_payload(proto))


('INSBI1', [('INSBI188022073', 'jhpicJqUN', 'yKEesKMCX,GjJFjdxaJ'), ('INSBI180433362', 'PvdkdMBYe', 'yKEesKMCX,GjJFjdxaJ')])
('INSBI6', [('INSBI652587880', 'XqMkSHEva', 'yKEesKMCX,GjJFjdxaJ'), ('INSBI657731628', 'hNQKABNvb', 'yKEesKMCX,GjJFjdxaJ')])
('INSBI5', [('INSBI585529195', 'xbbjDjkVx', 'yKEesKMCX,GjJFjdxaJ'), ('INSBI577879924', 'noNLOvNIo', 'yKEesKMCX,GjJFjdxaJ')])
('INSBI7', [('INSBI759226449', 'hthEUEqmx', 'yKEesKMCX,GjJFjdxaJ')])
('INSBI2', [('INSBI230887749', 'CajgKklNk', 'yKEesKMCX,GjJFjdxaJ')])
('INSBI4', [('INSBI481260153', 'ScQZUIHEv', 'yKEesKMCX,GjJFjdxaJ')])
('INSBI3', [('INSBI304740746', 'tadoHnbqL', 'yKEesKMCX,GjJFjdxaJ')])


In [81]:
query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records` LIMIT 10'



def cross_join(left, rights):
    for x in rights:
        yield (left['id'],left['name'],x)
        
def insert_key(x):
    return (x[0][0:6],(x[0],x[1],x[2]))

class make_api_call(beam.DoFn):
    
    def process(self, element):
        # importing the requests library
        import requests
        import json
        import urllib.request
        
        url_post = 'http://35.192.38.85:80/hello/post'
        data_dict={}
        data_dict['customer_list']=element
        post_response = requests.post(url_post, data = data_dict)
        
        yield element[1] #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator


from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

#p = beam.Pipeline(options=options)

with beam.Pipeline(options=options) as p:
    #with beam.Pipeline(options=options) as p:
    main_input = (p|beam.Create([('INSBI652587880', 'XqMkSHEva', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI759226449', 'hthEUEqmx', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI188022073', 'jhpicJqUN', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI230887749', 'CajgKklNk', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI585529195', 'xbbjDjkVx', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI180433362', 'PvdkdMBYe', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI481260153', 'ScQZUIHEv', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI657731628', 'hNQKABNvb', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI577879924', 'noNLOvNIo', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI304740746', 'tadoHnbqL', 'yKEesKMCX,GjJFjdxaJ')
                                 ]))
    key_insert = main_input|"Key Insertion">>beam.Map(insert_key)| 'Group into batches' >> beam.GroupIntoBatches(2)|'Make an External API Call' >> beam.ParDo(make_api_call())|beam.Map(print)

/tmp/ipykernel_3444/2262143131.py:47: FutureWarning: GroupIntoBatches is experimental.
  key_insert = main_input|"Key Insertion">>beam.Map(insert_key)| 'Group into batches' >> beam.GroupIntoBatches(2)|'Make an External API Call' >> beam.ParDo(make_api_call())|beam.Map(print)
/jupyter/.kernels/apache-beam-2.44.0/lib/python3.8/site-packages/apache_beam/transforms/util.py:924: FutureWarning: GroupIntoBatches is experimental.
  return GroupIntoBatches(*_GroupIntoBatchesParams.parse_payload(proto))


[('INSBI188022073', 'jhpicJqUN', 'yKEesKMCX,GjJFjdxaJ'), ('INSBI180433362', 'PvdkdMBYe', 'yKEesKMCX,GjJFjdxaJ')]
[('INSBI652587880', 'XqMkSHEva', 'yKEesKMCX,GjJFjdxaJ'), ('INSBI657731628', 'hNQKABNvb', 'yKEesKMCX,GjJFjdxaJ')]
[('INSBI585529195', 'xbbjDjkVx', 'yKEesKMCX,GjJFjdxaJ'), ('INSBI577879924', 'noNLOvNIo', 'yKEesKMCX,GjJFjdxaJ')]
[('INSBI759226449', 'hthEUEqmx', 'yKEesKMCX,GjJFjdxaJ')]
[('INSBI230887749', 'CajgKklNk', 'yKEesKMCX,GjJFjdxaJ')]
[('INSBI481260153', 'ScQZUIHEv', 'yKEesKMCX,GjJFjdxaJ')]
[('INSBI304740746', 'tadoHnbqL', 'yKEesKMCX,GjJFjdxaJ')]


In [99]:
query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records` LIMIT 10'



def cross_join(left, rights):
    for x in rights:
        yield (left['id'],left['name'],x)
        
def insert_key(x):
    return (x[0][0:6],(x[0],x[1],x[2]))

class make_api_call(beam.DoFn):
    
    def process(self, element):
        # importing the requests library
        import requests
        import json
        import urllib.request
        
        #import urllib
        #import urllib2
        
        list_c_id = []
        list_c_name = []
        
        for x in element[1]:
            list_c_id.append(x[0])
            list_c_name.append(x[1])
        
        url_post = 'http://35.192.38.85:80/hello/post'
        data_dict={
                 "customer_id":list_c_id,
                 "customer_name":list_c_name,
                 "sanctioned_name":element[1][0][2].split(',')
        }
        #data_dict['customer_id']=list_c_id
        #data_dict['customer_name']=list_c_name
        #data_dict['sanctioned_name']=element[1][0][2].split(',')
        
        url_post = 'http://35.192.38.85:80/hello/post'
        # json object getting serialised
        #json_object =  json.dumps(data_dict, indent = 8) #json.dumps(data_dict).encode('utf-8') 
        
        
        
        post_response = requests.post(url_post, data = data_dict)
        
        yield data_dict #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator


from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

#p = beam.Pipeline(options=options)

with beam.Pipeline(options=options) as p:
    #with beam.Pipeline(options=options) as p:
    main_input = (p|beam.Create([('INSBI652587880', 'XqMkSHEva', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI759226449', 'hthEUEqmx', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI188022073', 'jhpicJqUN', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI230887749', 'CajgKklNk', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI585529195', 'xbbjDjkVx', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI180433362', 'PvdkdMBYe', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI481260153', 'ScQZUIHEv', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI657731628', 'hNQKABNvb', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI577879924', 'noNLOvNIo', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI304740746', 'tadoHnbqL', 'yKEesKMCX,GjJFjdxaJ')
                                 ]))
    key_insert = main_input|"Key Insertion">>beam.Map(insert_key)| 'Group into batches' >> beam.GroupIntoBatches(2)|'Make an External API Call' >> beam.ParDo(make_api_call())|beam.Map(print)


/tmp/ipykernel_3444/4201210615.py:70: FutureWarning: GroupIntoBatches is experimental.
  key_insert = main_input|"Key Insertion">>beam.Map(insert_key)| 'Group into batches' >> beam.GroupIntoBatches(2)|'Make an External API Call' >> beam.ParDo(make_api_call())|beam.Map(print)
/jupyter/.kernels/apache-beam-2.44.0/lib/python3.8/site-packages/apache_beam/transforms/util.py:924: FutureWarning: GroupIntoBatches is experimental.
  return GroupIntoBatches(*_GroupIntoBatchesParams.parse_payload(proto))


{'customer_id': ['INSBI188022073', 'INSBI180433362'], 'customer_name': ['jhpicJqUN', 'PvdkdMBYe'], 'sanctioned_name': ['yKEesKMCX', 'GjJFjdxaJ']}
{'customer_id': ['INSBI652587880', 'INSBI657731628'], 'customer_name': ['XqMkSHEva', 'hNQKABNvb'], 'sanctioned_name': ['yKEesKMCX', 'GjJFjdxaJ']}
{'customer_id': ['INSBI585529195', 'INSBI577879924'], 'customer_name': ['xbbjDjkVx', 'noNLOvNIo'], 'sanctioned_name': ['yKEesKMCX', 'GjJFjdxaJ']}
{'customer_id': ['INSBI759226449'], 'customer_name': ['hthEUEqmx'], 'sanctioned_name': ['yKEesKMCX', 'GjJFjdxaJ']}
{'customer_id': ['INSBI230887749'], 'customer_name': ['CajgKklNk'], 'sanctioned_name': ['yKEesKMCX', 'GjJFjdxaJ']}
{'customer_id': ['INSBI481260153'], 'customer_name': ['ScQZUIHEv'], 'sanctioned_name': ['yKEesKMCX', 'GjJFjdxaJ']}
{'customer_id': ['INSBI304740746'], 'customer_name': ['tadoHnbqL'], 'sanctioned_name': ['yKEesKMCX', 'GjJFjdxaJ']}


In [115]:

query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records` LIMIT 20'


'''
def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))
'''
'''
def cross_join(left, rights):
    for x in rights:
        yield (left, x)
'''        
        
'''        
def singleton_cross_join(left, rights):
    return (left, rights)
'''

def cross_join(left, rights):
    for x in rights:
        yield (left['id'],left['name'],x)
        
def insert_key(x):
    return (x[0][0:6],(x[0],x[1],x[2]))



class make_api_call(beam.DoFn):
    
    def process(self, element):
        # importing the requests library
        import requests
        import json
        import urllib.request
        
        #import urllib
        #import urllib2
        
        list_c_id = []
        list_c_name = []
        
        for x in element[1]:
            list_c_id.append(x[0])
            list_c_name.append(x[1])
        
        url_post = 'http://35.192.38.85:80/hello/post'
        data_dict={
                 "customer_id":list_c_id,
                 "customer_name":list_c_name,
                 "sanctioned_name":element[1][0][2].split(',')
        }
        #data_dict['customer_id']=list_c_id
        #data_dict['customer_name']=list_c_name
        #data_dict['sanctioned_name']=element[1][0][2].split(',')
        
        url_post = 'http://35.192.38.85:80/hello/post'      
        post_response = requests.post(url_post, data = data_dict)
        x = post_response.text
        row = dict(zip(('Message','id'),(x,0)))
        yield row #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator

        
from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery
import apache_beam as beam 

p = beam.Pipeline(options=options)

main_input = (p|beam.io.ReadFromBigQuery(query=query,use_standard_sql=True ))#|beam.Map(insert_key)#|beam.Map(print))    
side_input = (p|"Side Input From GCS" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/sanctioned_customer.txt'))
enriched = (main_input|"Cross join">>beam.FlatMap(cross_join, rights=beam.pvalue.AsIter(side_input)))#|beam.Map(print) 
key_insert = enriched|"Key Insertion">>beam.Map(insert_key)#|beam.Map(print)
batches = key_insert|'Group into batches' >> beam.GroupIntoBatches(5)#|beam.Map(print)
process = batches|'Make an External API Call' >> beam.ParDo(make_api_call())|"Write to Bigquery">>beam.io.Write(beam.io.WriteToBigQuery('my-table-message-third',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='Message:STRING,id:INTEGER',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )
    


pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_3444/834015840.py:72: FutureWarning: GroupIntoBatches is experimental.
  batches = key_insert|'Group into batches' >> beam.GroupIntoBatches(5)#|beam.Map(print)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [114]:
query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records` LIMIT 10'



def cross_join(left, rights):
    for x in rights:
        yield (left['id'],left['name'],x)
        
def insert_key(x):
    return (x[0][0:6],(x[0],x[1],x[2]))

class make_api_call(beam.DoFn):
    
    def process(self, element):
        # importing the requests library
        import requests
        import json
        import urllib.request
        
        #import urllib
        #import urllib2
        
        list_c_id = []
        list_c_name = []
        
        for x in element[1]:
            list_c_id.append(x[0])
            list_c_name.append(x[1])
        
        url_post = 'http://35.192.38.85:80/hello/post'
        data_dict={
                 "customer_id":list_c_id,
                 "customer_name":list_c_name,
                 "sanctioned_name":element[1][0][2].split(',')
        }
        #data_dict['customer_id']=list_c_id
        #data_dict['customer_name']=list_c_name
        #data_dict['sanctioned_name']=element[1][0][2].split(',')
        
        url_post = 'http://35.192.38.85:80/hello/post'
        # json object getting serialised
        #json_object =  json.dumps(data_dict, indent = 8) #json.dumps(data_dict).encode('utf-8') 
        
        
        
        post_response = requests.post(url_post, data = data_dict)
        x = post_response.text
        row = dict(zip(('Message','id'),(x,0)))
        yield row #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator


from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

#p = beam.Pipeline(options=options)

with beam.Pipeline(options=options) as p:
    #with beam.Pipeline(options=options) as p:
    main_input = (p|beam.Create([('INSBI652587880', 'XqMkSHEva', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI759226449', 'hthEUEqmx', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI188022073', 'jhpicJqUN', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI230887749', 'CajgKklNk', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI585529195', 'xbbjDjkVx', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI180433362', 'PvdkdMBYe', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI481260153', 'ScQZUIHEv', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI657731628', 'hNQKABNvb', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI577879924', 'noNLOvNIo', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI304740746', 'tadoHnbqL', 'yKEesKMCX,GjJFjdxaJ')
                                 ]))
    key_insert = main_input|"Key Insertion">>beam.Map(insert_key)| 'Group into batches' >> beam.GroupIntoBatches(2)|'Make an External API Call' >> beam.ParDo(make_api_call())|beam.Map(print)


/tmp/ipykernel_3444/3183263075.py:71: FutureWarning: GroupIntoBatches is experimental.
  key_insert = main_input|"Key Insertion">>beam.Map(insert_key)| 'Group into batches' >> beam.GroupIntoBatches(2)|'Make an External API Call' >> beam.ParDo(make_api_call())|beam.Map(print)
/jupyter/.kernels/apache-beam-2.44.0/lib/python3.8/site-packages/apache_beam/transforms/util.py:924: FutureWarning: GroupIntoBatches is experimental.
  return GroupIntoBatches(*_GroupIntoBatchesParams.parse_payload(proto))


{'Message': '<h1>invalid credentials!</h1>', 'id': 0}
{'Message': '<h1>invalid credentials!</h1>', 'id': 0}
{'Message': '<h1>invalid credentials!</h1>', 'id': 0}
{'Message': '<h1>invalid credentials!</h1>', 'id': 0}
{'Message': '<h1>invalid credentials!</h1>', 'id': 0}
{'Message': '<h1>invalid credentials!</h1>', 'id': 0}
{'Message': '<h1>invalid credentials!</h1>', 'id': 0}


In [121]:

query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records`'


'''
def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))
'''
'''
def cross_join(left, rights):
    for x in rights:
        yield (left, x)
'''        
        
'''        
def singleton_cross_join(left, rights):
    return (left, rights)
'''

def cross_join(left, rights):
    for x in rights:
        yield (left['id'],left['name'],x)
        
def insert_key(x):
    return (x[0][0:6],(x[0],x[1],x[2]))



class make_api_call(beam.DoFn):
    
    def process(self, element):
        # importing the requests library
        import requests
        import json
        import urllib.request
        
        #import urllib
        #import urllib2
        
        list_c_id = []
        list_c_name = []
        
        for x in element[1]:
            list_c_id.append(x[0])
            list_c_name.append(x[1])
        
        url_post = 'http://35.192.38.85:80/hello/post'
        data_dict={
                 "customer_id":list_c_id,
                 "customer_name":list_c_name,
                 "sanctioned_name":element[1][0][2].split(',')
        }
        #data_dict['customer_id']=list_c_id
        #data_dict['customer_name']=list_c_name
        #data_dict['sanctioned_name']=element[1][0][2].split(',')
        
        url_post = 'http://35.192.38.85:80/hello/post'      
        post_response = requests.post(url_post, data = data_dict)
        x = post_response.text
        row = dict(zip(('Message','id'),(x,0)))
        yield row #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator

        
from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery
import apache_beam as beam 

p = beam.Pipeline(options=options)

main_input = (p|beam.io.ReadFromBigQuery(query=query,use_standard_sql=True ))#|beam.Map(insert_key)#|beam.Map(print))    
side_input = (p|"Side Input From GCS" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/sanctioned_customer.txt'))
enriched = (main_input|"Cross join">>beam.FlatMap(cross_join, rights=beam.pvalue.AsIter(side_input)))#|beam.Map(print) 
key_insert = enriched|"Key Insertion">>beam.Map(insert_key)#|beam.Map(print)
batches = key_insert|'Group into batches' >> beam.GroupIntoBatches(8000)#|beam.Map(print)
process = batches|'Make an External API Call' >> beam.ParDo(make_api_call())|"Write to Bigquery">>beam.io.Write(beam.io.WriteToBigQuery('my-table-message-third',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='Message:STRING,id:INTEGER',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )
    


pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_3444/2343147345.py:72: FutureWarning: GroupIntoBatches is experimental.
  batches = key_insert|'Group into batches' >> beam.GroupIntoBatches(3000)#|beam.Map(print)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [133]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import WorkerOptions
import apache_beam as beam
import time
from apache_beam.runners import DataflowRunner


options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True
options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#number of workers
options.view_as(WorkerOptions).num_workers = 2

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)


########################################################################  With A Single Worker    ############################################################################


query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records`'



def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))

def makeRow(x):
    row = dict(zip(('id','name'),x))
    return row

class make_api_call(beam.DoFn):
    
    def process(self, element, right):
        # importing the requests library
        import requests
        import json
        import urllib.request
        #from kafka import KafkaProducer
        
        #producer = KafkaProducer(bootstrap_servers=['34.28.118.32:9094'], api_version=(0, 10))
        
        #content = urllib.request.urlopen('http://35.239.64.67:32726/').read().decode('utf-8')
        #row = dict(zip(('message'),content))
        
        
        
        # The API endpoint
        #url = "http://35.239.64.67:32726/"
        url = "https://jsonplaceholder.typicode.com/posts/1"

        # A GET request to the API
        response = requests.get(url)

        # Print the response
        response_json = response.json()
        #producer.send('my-topic', json.dumps(response_json).encode('utf-8'))
        #if producer is not None:
                    #producer.close()

        length = len(element[1])
        x = response_json['userId']
        x1 = response_json['id']
        x2 = response_json['title']
        x3 = response_json['body']
        #x=1
        #x1=1
        #x2='string'
        #x3='strong'
        
        row = dict(zip(('userId','id','title','body','length'),(x,x1,x2,x3,length)))
        yield row #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator
          

from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 

p = beam.Pipeline(options=options)

#with beam.Pipeline(options=options) as p:
side_input = (p|"Side Input From GCS" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/sanctioned_customer.txt'))
read_requests =(p| 
    beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )
    |beam.Map(insert_key)
    |'Group into batches' >> beam.GroupIntoBatches(50000)
    |'Make an External API Call' >> beam.ParDo(make_api_call(),beam.pvalue.AsIter(side_input))
    |"Write to Bigquery">>beam.io.Write(beam.io.WriteToBigQuery(        'my-table-message-second',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='userId:INTEGER, id:INTEGER,title:STRING,body:STRING,length:INTEGER',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )
    )
    #|beam.Map(print))
               
pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_3444/3366851488.py:113: FutureWarning: GroupIntoBatches is experimental.
  |'Group into batches' >> beam.GroupIntoBatches(50000)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [151]:
query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records` LIMIT 10'



def cross_join(left, rights):
    for x in rights:
        yield (left['id'],left['name'],x)
        
def insert_key(x):
    return (x[0][0:6],(x[0],x[1],x[2]))

class make_api_call(beam.DoFn):
    
    def process(self, element, side_input):
        
        # importing the requests library
        import requests
        import json
        import urllib.request
        
        #import urllib
        #import urllib2
        
        list_c_id = []
        list_c_name = []
        
        for x in element[1]:
            list_c_id.append(x[0])
            list_c_name.append(x[1])
        
        url_post = 'http://35.192.38.85:80/hello/post'
        data_dict={
                 "customer_id":list_c_id,
                 "customer_name":list_c_name,
                 "sanctioned_name":side_input.split(',')
        }
        #data_dict['customer_id']=list_c_id
        #data_dict['customer_name']=list_c_name
        #data_dict['sanctioned_name']=element[1][0][2].split(',')
        
        url_post = 'http://35.192.38.85:80/hello/post'      
        post_response = requests.post(url_post, data = data_dict)
        x = post_response.text
        row = dict(zip(('Message','id'),(x,0)))
        yield row #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator


def log_element(element):
    logging.info(element)
    return element

from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 
import logging

p = beam.Pipeline(options=options)

#with beam.Pipeline(options=options) as p:
side_input = (p|"Side Input From GCS" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/sanctioned_customer.txt'))
#with beam.Pipeline(options=options) as p:
#side_input = (p|beam.Create(['yKEesKMCX,GjJFjdxaJ,XqMkSHEva,ScQZUIHEv']))
main_input = (p|beam.Create([('INSBI652587880', 'XqMkSHEva', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI759226449', 'hthEUEqmx', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI188022073', 'jhpicJqUN', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI230887749', 'CajgKklNk', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI585529195', 'xbbjDjkVx', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI180433362', 'PvdkdMBYe', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI481260153', 'ScQZUIHEv', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI657731628', 'hNQKABNvb', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI577879924', 'noNLOvNIo', 'yKEesKMCX,GjJFjdxaJ'),
                                 ('INSBI304740746', 'tadoHnbqL', 'yKEesKMCX,GjJFjdxaJ')
                                 ]))
key_insert = main_input|"Key Insertion">>beam.Map(insert_key)| 'Group into batches' >> beam.GroupIntoBatches(2)#|beam.Map(print)
api_call =  key_insert|'Make an External API Call' >> beam.ParDo(make_api_call(),beam.pvalue.AsSingleton(side_input))|beam.Map(log_element)#beam.Map(print)
#|'Make an External API Call' >> beam.ParDo(make_api_call())|beam.Map(print)
    
pipeline_result = DataflowRunner().run_pipeline(p, options=options).wait_until_finish()

/tmp/ipykernel_3444/3764640179.py:74: FutureWarning: GroupIntoBatches is experimental.
  key_insert = main_input|"Key Insertion">>beam.Map(insert_key)| 'Group into batches' >> beam.GroupIntoBatches(2)#|beam.Map(print)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [150]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import WorkerOptions
import apache_beam as beam
import time
from apache_beam.runners import DataflowRunner


options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True
options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#number of workers
options.view_as(WorkerOptions).num_workers = 2

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)


########################################################################  With A Single Worker    ############################################################################


query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records`'



def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))

def makeRow(x):
    row = dict(zip(('id','name'),x))
    return row

class make_api_call(beam.DoFn):
    
    def process(self, element, right):
        # importing the requests library
        import requests
        import json
        import urllib.request
        #from kafka import KafkaProducer
        
        #producer = KafkaProducer(bootstrap_servers=['34.28.118.32:9094'], api_version=(0, 10))
        
        #content = urllib.request.urlopen('http://35.239.64.67:32726/').read().decode('utf-8')
        #row = dict(zip(('message'),content))
        
        
        
        # The API endpoint
        #url = "http://35.239.64.67:32726/"
        url = "https://jsonplaceholder.typicode.com/posts/1"

        # A GET request to the API
        response = requests.get(url)

        # Print the response
        response_json = response.json()
        #producer.send('my-topic', json.dumps(response_json).encode('utf-8'))
        #if producer is not None:
                    #producer.close()

        length = len(element[1])
        x = response_json['userId']
        x1 = response_json['id']
        x2 = response_json['title']
        x3 = response_json['body']
        #x=1
        #x1=1
        #x2='string'
        #x3='strong'
        
        row = dict(zip(('userId','id','title','body','length'),(x,x1,x2,x3,length)))
        yield row #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator
          
def log_element(element):
    logging.info(element)
    return element
            
            
from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 
import logging

p = beam.Pipeline(options=options)

#with beam.Pipeline(options=options) as p:
side_input = (p|"Side Input From GCS" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/sanctioned_customer.txt'))
read_requests =(p| 
    beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )
    |beam.Map(insert_key)
    |'Group into batches' >> beam.GroupIntoBatches(50000)
    |'Make an External API Call' >> beam.ParDo(make_api_call(),beam.pvalue.AsSingleton(side_input))
    |"Write to Bigquery">>beam.io.Write(beam.io.WriteToBigQuery(        'my-table-message-second',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='userId:INTEGER, id:INTEGER,title:STRING,body:STRING,length:INTEGER',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
                                                       )
    )
    #|beam.Map(print))
               
pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_3444/1345512093.py:118: FutureWarning: GroupIntoBatches is experimental.
  |'Group into batches' >> beam.GroupIntoBatches(50000)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip


In [155]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import WorkerOptions
import apache_beam as beam
import time
from apache_beam.runners import DataflowRunner


options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True
options.view_as(GoogleCloudOptions).enable_streaming_engine = True
#options.view_as(PipelineOptions).experiments = 'use_runner_v2' the log says it is by deafult

#number of workers
options.view_as(WorkerOptions).num_workers = 2

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)


########################################################################  With A Single Worker    ############################################################################


query='SELECT id,name FROM `mimetic-parity-378803.gcpdataset.my-table-customer-records`'



def insert_key(x):
    return (x['id'][0:6],(x['id'],x['name']))

def makeRow(x):
    row = dict(zip(('id','name'),x))
    return row

class make_api_call(beam.DoFn):
    
    def process(self, element, side_input):
        # importing the requests library
        import requests
        import json
        import urllib.request
        
        #import urllib
        #import urllib2
        
        list_c_id = []
        list_c_name = []
        
        for x in element[1]:
            list_c_id.append(x[0])
            list_c_name.append(x[1])
        
        url_post = 'http://35.192.38.85:80/hello/post'
        data_dict={
                 "customer_id":list_c_id,
                 "customer_name":list_c_name,
                 "sanctioned_name":side_input.split(',')
        }
        #data_dict['customer_id']=list_c_id
        #data_dict['customer_name']=list_c_name
        #data_dict['sanctioned_name']=element[1][0][2].split(',')
        length = len(element[1])
        url_post = 'http://35.192.38.85:80/hello/post'      
        post_response = requests.post(url_post, data = data_dict)
        x = post_response.text
        row = dict(zip(('Message','id','Batchsize'),(x,0,length)))
        yield row #A return here was not working, please remember, from error message [row[0].json cannot be done], so the write to bigquery expects an interator
          
def log_element(element):
    logging.info(element)
    return element
            
            
from apache_beam.io.gcp.bigquery import  ReadFromBigQueryRequest, ReadAllFromBigQuery

import apache_beam as beam 
import logging

p = beam.Pipeline(options=options)

#with beam.Pipeline(options=options) as p:
side_input = (p|"Side Input From GCS" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/sanctioned_customer.txt'))
read_requests =(p| 
    beam.io.ReadFromBigQuery(query=query,use_standard_sql=True )
    |beam.Map(insert_key)
    |'Group into batches' >> beam.GroupIntoBatches(10000)
    |'Make an External API Call' >> beam.ParDo(make_api_call(),beam.pvalue.AsSingleton(side_input))
    |"Write to Bigquery">>beam.io.Write(beam.io.WriteToBigQuery('my-table-message-fourth',
                                                                        dataset='gcpdataset',
                                                                        project='mimetic-parity-378803',
                                                                        schema='Message:STRING,id:INTEGER,Batchsize:INTEGER',
                                                                        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED, #If table does not exist before, this code throws error
                                                                        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
                                                                        
                                                                      )
    )
               )
    #|beam.Map(print))
               
pipeline_result = DataflowRunner().run_pipeline(p, options=options)

/tmp/ipykernel_3444/103542436.py:110: FutureWarning: GroupIntoBatches is experimental.
  |'Group into batches' >> beam.GroupIntoBatches(10000)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: /jupyter/.kernels/apache-beam-2.44.0/bin/python -m pip install --upgrade pip
